In [1]:
import numpy as np
import os
import random

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import layers, rnn, seq2seq
from tqdm import tqdm

In [2]:
import fasttext as fs

In [3]:
from data_generator import NLUDataGenerator
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
from tensorflow.contrib.legacy_seq2seq import rnn_decoder

In [48]:
DataGenDev.vocab_size

713

In [49]:
DataGen.vocab_size

822

In [14]:
PAD = 0
EOS = 1

TimeMajor = False
batch_size = 32
encoder_max_time = 64

DataGenDev = NLUDataGenerator('../data/dev/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', None, seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

DataGen = NLUDataGenerator('../data/train/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', DataGenDev.vocab, seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

vocab_size = len(DataGen.vocab)
input_embedding_size = 100
# encoder_hidden_units = 128
# decoder_hidden_units = encoder_hidden_units * 2

len(diff):  175
tmp len:  175
before:  711
fin:  886


In [6]:
x, m1, m2, _lens = next(DataGen)
for x_, m1_, m2_ in zip(x, m1, m2):
    print(DataGen.decode_sentence(x_))
    print(DataGen.decode_acts(m2_))
    print(DataGen.decode_slots(m1_))
    print("-----")

['i', 'dont', 'care', 'about', 'price']
['O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O']
-----
['music', 'im', 'looking', 'for', 'a', 'expensive', 'priced', 'restaurant', 'in', 'the', 'centre', 'part', 'of', 'town', 'please']
['O', 'O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O']
-----
['is', 'there', 'on', 'serving', 'chineese']
['O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O']
-----
['what', 'price', 'ran']
['O', 'B-request', 'I-request']
['O', 'B-pricerange', 'I-pricerange']
-----
['id', 'like', 'to', 'find', 'a', 'restaurant', 'serving', 'swiss', 'food']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-food', 'O']
-----
['i', 'dont', 'care', 'about', 'the', 'price', 'range']
['O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
['id', 'like', 'a', 'cheap', 'korean', 'restaurant']
['O', 

In [7]:
slots_number = len(DataGen.slots_encode)
acts_number = len(DataGen.acts_encode)
print("slots_number: ", slots_number)
print("acts_number: ", acts_number)
print("vocab_size: ", DataGen.vocab_size)

slots_number:  26
acts_number:  18
vocab_size:  886


In [15]:
# MAKING EMBEDDING MATRIX

fs_model = fs.load_model("./model.bin")
embedd = np.zeros((vocab_size, input_embedding_size), dtype=np.float32)
for k, v in DataGen.vocab.items():
    embedd[v] = fs_model[k]
embedd[0] = [0]*100

In [18]:
tf.reset_default_graph()

inputs = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask1 = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask2 = tf.placeholder(shape=(None, None), dtype=tf.int32)
lens = tf.placeholder(shape=(None,), dtype=tf.int32)

embeddings = tf.get_variable('embeddings', initializer=embedd, dtype=tf.float32, trainable=False)

encoded = tf.nn.embedding_lookup(embeddings, inputs)
cell = LSTMCell(input_embedding_size)

outputs, _ = tf.nn.dynamic_rnn(cell, encoded, sequence_length=lens, dtype=tf.float32)


seq_mask = tf.to_float(tf.greater(inputs, PAD))
with tf.variable_scope('lm-reg'):
    first_words = tf.slice(inputs, [0, 1], [-1, -1])
    target = tf.pad(first_words, [[0, 0], [0, 1]])
    nwp = layers.fully_connected(outputs, vocab_size, activation_fn=None)
    lm_loss = seq2seq.sequence_loss(nwp, target, seq_mask)

with tf.variable_scope('mask-pred'):
    m1p = layers.fully_connected(outputs, slots_number, activation_fn=None)
    m2p = layers.fully_connected(outputs, acts_number, activation_fn=None)
    
    prediction_m1 = tf.argmax(tf.nn.softmax(m1p), axis = 2)
    prediction_m2 = tf.argmax(tf.nn.softmax(m2p), axis = 2)
    
    m1_loss = seq2seq.sequence_loss(m1p, mask1, seq_mask)
    m2_loss = seq2seq.sequence_loss(m2p, mask2, seq_mask)
    
    
lr = tf.Variable(0.01, trainable=False)
opt = tf.train.GradientDescentOptimizer(lr)
train_op = opt.minimize(lm_loss + m1_loss + m2_loss)

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    avg_loss = -np.ones((3, ))
    alpha = 0.9
    
    pbar = tqdm(range(1000))
    for step in pbar:
        x, m1, m2, _lens = next(DataGen)
        
        res = sess.run([lm_loss, m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
        res = np.array(res[:-1])
        
        if avg_loss[0] < 0:
            avg_loss = res
        
        avg_loss = alpha * avg_loss + (1-alpha) * res
        
        pbar.set_description(' '.join('{:.2f}'.format(_) for _ in avg_loss))
        
        if(step%400 == 0):
            
            x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
            print("AVG_loss: ", avg_loss)
            print("step: ", step)
            
            N = 3 # how many examples show to test:
            pred_1, pred_2 = sess.run([prediction_m1, prediction_m2], {inputs: x_dev[:N], 
                                                              mask1: m1_dev[:N], 
                                                              mask2: m2_dev[:N], 
                                                              lens: _lens_dev[:N]})
            
            for x_, m1_, m2_, p1, p2 in zip(x_dev[:N], m1_dev[:N], m2_dev[:N], pred_1, pred_2):

                print("INPUT: ", DataGen.decode_sentence(x_))
                print("m2_tar: ", DataGen.decode_acts(m2_))
                print("m2_pred: ", DataGen.decode_acts(p2))                
                
                print("-----")
                
                print("m1_tar: ", DataGen.decode_slots(m1_))
                print("m1_pred: ", DataGen.decode_slots(p1))
                
                print("==========================")
            


6.79 3.31 2.79:   1%|          | 6/1000 [00:00<01:32, 10.77it/s]

AVG_loss:  [ 6.7941494   3.37402844  2.85030293]
step:  0
INPUT:  ['adddress', 'telephone']
m2_tar:  ['B-inform', 'B-inform']
m2_pred:  ['I-bye', 'I-bye', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['B-addr', 'B-phone']
m1_pred:  ['B-name', 'I-addr', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['and', 'telephone', 'nubmer']
m2_tar:  ['O', 'B-inform', 'O']
m2_pred:  ['I-bye', 'I-bye', 'I-bye', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'B-phone', 'O']
m1_pred:  ['I-food', 'B-name', 'I-addr', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['doesnt', 'matter']
m2_tar:  ['O', 'O']
m2_pred:  ['O', 'B-hello', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O']
m1_pred:  ['I-hello', 'I-hello', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



5.52 1.24 1.01:  41%|████      | 406/1000 [00:08<00:12, 48.37it/s]

AVG_loss:  [ 5.54633522  1.29391539  1.05486703]
step:  400
INPUT:  ['chinese', 'food', 'in', 'dontcare', 'area']
m2_tar:  ['B-request', 'O', 'O', 'B-request', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['B-food', 'O', 'O', 'B-area', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['thank', 'you', 'goodbye']
m2_tar:  ['B-bye', 'B-bye', 'B-bye']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['B-bye', 'B-bye', 'B-bye']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['what', 'cost', 'is', 'this', 'venue']
m2_tar:  ['O', 'B-inform', 'O', 'O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'B-pricerange', 'O', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

4.87 1.09 0.89:  81%|████████  | 806/1000 [00:16<00:03, 49.41it/s]

AVG_loss:  [ 4.90570879  1.09427881  0.8916809 ]
step:  800
INPUT:  ['is', 'there', 'a', 'cheap', 'priced', 'restaurant', 'that', 'serves', 'caribbean', 'food']
m2_tar:  ['O', 'O', 'O', 'B-affirm', 'O', 'O', 'O', 'O', 'B-affirm', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'O', 'B-food', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['how', 'about', 'a', 'expensive', 'priced']
m2_tar:  ['O', 'O', 'O', 'B-request', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'B-pricerange', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['a', 'restaurant', 'that', 'serves', 'african', 'food']
m2_tar:  ['O', 'O', 'O', 'O', 'B-request', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['

4.83 1.01 0.81: 100%|██████████| 1000/1000 [00:20<00:00, 47.88it/s]


In [12]:
file_writer = tf.summary.FileWriter('../logdir/', sess.graph)

## Это чисто черновик, где происходит какая-то фигня с bi-LSTM

In [ ]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

encoder_targets = tf.placeholder(shape=(None), dtype=tf.int32, name='encoder_targets')
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [ ]:
encoder_cell_b = LSTMCell(encoder_hidden_units)
encoder_cell_f = LSTMCell(encoder_hidden_units, reuse=None)

In [ ]:
# encoder_cell = LSTMCell(encoder_hidden_units, reuse=tf.get_variable_scope().reuse)
# encoder_cell_f = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
encoder_bw_outputs),
(encoder_fw_final_state,
encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell_f,
                                cell_bw=encoder_cell_b,
                                inputs=encoder_inputs_embedded,
                                sequence_length=encoder_inputs_length,
                                dtype=tf.float32, time_major=TimeMajor))
    

In [ ]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h)

In [ ]:
from tensorflow.contrib import layers
next_word_pred = layers.fully_connected(encoder_outputs, vocab_size, ...)



mask1_pred = layers.fully_connected(encoder_outputs, mask1_vocab, ...)
mask2_pred = layers.fully_connected(encoder_outputs, mask2_vocab, ...)


In [ ]:
encoder_final_state_h

In [ ]:
W = tf.Variable(tf.random_uniform([encoder_hidden_units*2, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
logits = tf.add(tf.matmul(encoder_final_state_h, W), b)
# encoder_logits = tf.reshape(encoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [ ]:
logits

In [ ]:
encoder_targets

In [ ]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = encoder_targets)